In [1]:
#-------------Imports--------------
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as optimize
from sklearn.preprocessing import StandardScaler
import math

In [2]:
#-----------Load Dataset-----------

path_train = "fashion_train.npy"
path_test = "fashion_test.npy"

train = np.load(path_train)
test = np.load(path_test)

#Split the training and test data into features and labels
X_train = train[:,:784]
y_train = train[:,784]

X_test = test[:,:784]
y_test = test[:,784]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10000, 784)
(10000,)
(5000, 784)
(5000,)


## LDA

We will be using the first 2 linear discriminant variables as features for the Naive Bayes

In [3]:
def get_LDA_components(X,y):
    
    def get_Sw(X, y):
    
        N = X.shape[1] #number of features
        S_w = np.zeros((N,N))
        class_labels = np.unique(y)
        c = class_labels.shape[0] #number of classes

        #calculate scatter matrix for each class
        for class_ in range(c):

            S_i = np.zeros((N,N))
            class_subset = X[y == class_] #get rows which are a part of the current class
            mean_vector = (np.mean(class_subset, axis=0)).reshape(N, 1) #vector m_i containing
            #means of all features in class i

            for row_idx in range(class_subset.shape[0]):

                x = (class_subset[row_idx, :]).reshape(N, 1)
                S_i += (np.dot((x - mean_vector), np.transpose(x - mean_vector))) #apply formula for within class scatter matrix

            S_w += S_i

        return S_w
    #--------------Compute Between Class Scatter Matrix---------------
    def get_Sb(X, y):
    
        N = X.shape[1] #number of features
        m = (np.mean(X, axis=0)).reshape(N,1) #overall mean
        S_b = np.zeros((N,N))
        class_labels = np.unique(y)
        c = class_labels.shape[0] #number of classes

        for class_ in range(c):

            class_subset = X[y == class_]
            n_rows = class_subset.shape[0] #get number of rows which are a part of the current class
            mean_vector = (np.mean(class_subset, axis=0)).reshape(N, 1) #vector m_i containing
            #means of all features in class i
            S_b += n_rows * ((mean_vector - m).dot((mean_vector - m).T)) #apply formula for between class scatter matrix

        return S_b
    
    def get_linear_discriminants(S_w, S_b):
    
        # calculate the eigenvectors and eigenvalues of the matrix ((S_w)^-1)(S_b)
        eig_vals, eig_vecs = np.linalg.eig((np.linalg.inv(S_w)).dot(S_b))

        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))] #create a list of corresponding
        #eigenvectors and eigenvalues
        eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
        #sort the list by the eigenvalues in decreasing order

        return eig_pairs
    
    N = X.shape[1] #get number of features
    S_w = get_Sw(X, y) #get within class scatter matrix
    S_b = get_Sb(X, y) #get between class scatter matrix
    
    sorted_eigenvecs = get_linear_discriminants(S_w, S_b) #get linear discriminants sorted by
    #variance explained in descending order (most descriptive first)
    
    #get first 2 linear discriminants
    W = np.hstack((sorted_eigenvecs[0][1].reshape(N,1), sorted_eigenvecs[1][1].reshape(N,1)))
    
    #transform the samples onto the new subspace
    #transformed = X.dot(W)
    
    return W

In [4]:
W = get_LDA_components(X_train,y_train)

In [5]:
X_lda = X_train.dot(W)

## Naive Bayes

### Priors

Get probability of a sample being from a certain class from the distribution of classes in the training set<br><br>

To calculate the class priors, we'll be using the following formula:<br>

$$p(C_k) = \frac{n_k}{n}$$<br>

Where:<br>
$p(C_k)$ - prior probability<br>
$n_k$ - number of training samples of a particular class<br>
$n$ - total number of training samples<br>

In [6]:
def get_priors(X, y):
    
    n_rows = X.shape[0]
    classes, counts = np.unique(y, return_counts=True) # return how many samples there are of each class
    
    # prior probability of a sample being from a certain class
    priors = {class_: count / n_rows for class_, count in zip(classes, counts)}
    
    return priors

In [7]:
priors = get_priors(X_lda, y_train)

In [8]:
priors

{0: 0.2033, 1: 0.1947, 2: 0.2001, 3: 0.2005, 4: 0.2014}

## Feature PMF

Get probability of a feature being in different value ranges.<br>
To estimate these probabilities we use histograms returning the value range (calculated based on the hyperparameter h - the number of bins) and the number of samples which contain the corresponding feature in that range - so we divide this sample count by the total number of samples to calculate the probability.<br>
We will be using this function to calculate the conditional probabilities, finding the PMF of each feature given that it belongs to each of the possible classes.<br><br>

Note: It's possible that some resulting bins may be empty if there are no samples with the corresponding feature in the value range of that bin - case in which the probability of that value range will be 0.<br>
This means that if any sample has the corresponding feature in that value range with an empty bin it will be immediately disqualified from being from that class because the conditional probability of that feature will be 0, so the product of conditional probabilities will be 0. To combat this we added a small number instead of a flat 0 - so an unlikely feature is still heavily penalized, but not immediately disqualified from being from that class.

In [9]:
def estimate_pmf_hist(X, h):

    # return number of samples in each bin and edge values of the bin
    counts, edges = np.histogram(X, bins=h)
    
    # list of probabilities of a feature being from a certain value range
    cond = []
    
    # total number of samples
    total = np.sum(counts)
    
    for upper_bound, c in zip(edges[1:], counts):
        
        # get percentage of samples in bin and append ending point of the bin
        # and percentage to the conditionals list
        if c != 0:
            curr_cond = c / total
        
        # if a bin is not assigned any points (i.e there were no features in that value range)
        # assign it a very very low probability rather than a flat 0
        # chose to do this because a single PIXEL not in a yet encountered value range for that
        # feature for the given class immediately disqualifies that picture being from that class
        # otherwise, so instead give it a big penalty without immediately disqualifying it
        else:
            curr_cond = 0.001
            
        cond.append((upper_bound.real, curr_cond))
        
    return cond

## Conditional probabilities

Calculate the probability of each feature being in a certain value range given that it comes from each of the possible classes.<br><br>
We are returning them as a dictionary where each of the possible classes is a key, and the value of each of those classes is another dictionary, where each of the features(their index) is a key, and each of their values is a list containing tuples corresponding to each of the value intervals' upper bound, and the probability of the feature being in that range, given that it comes from its corresponding class.

In [10]:
def get_conditionals(X, y, h):
    
    N = X.shape[1] # number of features
    class_labels = np.unique(y)
    c = class_labels.shape[0] # number of classes
    
    # get conditional probabilities of features given the sample is from a certain class
    conditionals = {class_ : {} for class_ in class_labels}
    
    for class_ in range(c):
        
        # get features and labels corresponding to samples from the current class
        rows_subset = X[y == class_]
        
        for feature in range(N):
            
            features_subset = rows_subset[:, feature] # get vector of values of a certain feature in all samples
                                                      # of the current class
            
            # get probabilities of feature value ranges given the current class
            estimate = estimate_pmf_hist(features_subset, h)
            
            # Note: this should never happen if we cleaned the data set, but just in case for debugging
            if estimate == 0 or estimate == None:
                print(f"no bins at feature {feature} in class {class_}")
                
            conditionals[class_][feature] = estimate
            
    return conditionals

In [11]:
n_bins = 5
conditionals_lda = get_conditionals(X_lda, y_train, n_bins)
conditionals_lda

/Users/andrei/Library/Python/3.9/lib/python/site-packages/numpy/lib/histograms.py:841: ComplexWarning: Casting complex values to real discards the imaginary part
  indices = f_indices.astype(np.intp)


{0: {0: [(-5.95747735381325, 0.000983767830791933),
   (-3.626502487514723, 0.025086079685194294),
   (-1.2955276212161966, 0.49188391539596654),
   (1.035447245082329, 0.43433349729463844),
   (3.366422111380855, 0.04771273979340875)],
  1: [(-2.4288518255313014, 0.010329562223315297),
   (-0.41876769278369075, 0.15937038858829317),
   (1.5913164399639195, 0.6527299557304476),
   (3.6014005727115306, 0.16969995081160846),
   (5.611484705459141, 0.007870142646335464)]},
 1: {0: [(-0.5129972532639577, 0.004622496147919877),
   (4.041636692065667, 0.025166923472008218),
   (8.596270637395293, 0.11248073959938366),
   (13.150904582724916, 0.6373908577298408),
   (17.70553852805454, 0.22033898305084745)],
  1: [(-4.694712491619638, 0.007704160246533128),
   (-2.4261021583731903, 0.06420133538777606),
   (-0.1574918251267423, 0.8294812532100667),
   (2.1111185081197057, 0.08269131997945557),
   (4.379728841366154, 0.015921931176168466)]},
 2: {0: [(-4.896874831185922, 0.004997501249375313),

## Predict probabilities

For each sample, predict the probability of that sample being from each of the possible classes.<br><br>
To do so, for each class, we'll look at the probability of each of the features belonging to that class (the probability of that feature being in the value range it is given that it's from a certain class), and we'll take the product of those probabilities as the probability that our observed sample is from that class - this assumes that the features are all independent, which is why it's "naive".<br>
These are the class conditional probabilities, calculated with the following formula:<br>
$$p(C_k | x) \approx \prod_{j}^{m}p(x_j | C_k)$$<br>

Where:<br>
$p(C_k | x)$ - probability of sample being from a particular class given its features<br>
$p(x_j | C_k)$ - probability of a certain feature given that it comes from a certain class

After we have the probability of a sample being from each of the classes, we will also normalize those probabilities.

In [12]:
def predict_probabilities(X, cond, priors):
    
    N = X.shape[1] #number of features
    n_rows = X.shape[0]
    c = len(priors) #number of classes
    
    # matrix of class probabilities for each sample
    probs = np.zeros((n_rows, c))
    
    # for each sample
    for i in range(n_rows):
        
        # current sample
        x = X[i]
        
        # vector of class probabilities for the current sample
        class_probs = np.zeros((c, 1))
        
        # for each class
        for class_ in range(c):
            
            # get class prior
            prior = priors[class_]
            
            # initialize the probability of the sample belonging to that class
            # as the prior of that class
            class_prob = prior
            
            # for each feature
            for j in range(N):
                
                feature = x[j]
                
                # get probability of current feature
                # if it doesn't exist in the conditionals dictionary return a very very small probability
                curr_pmf = cond[class_][j]
                
                feature_prob = 0.001
                
                # find value interval in which the feature is and assign it
                # the conditional probability corresponding to that interval
                for upper_bound, cond_prob in curr_pmf:
                    
                    if upper_bound < feature:
                        continue
                    else:
                        feature_prob = cond_prob
                        break
                    
                # the probability of the sample belonging to the current class
                # is the probability of all of the feature values 
                class_prob *= feature_prob
                
                
            class_probs[class_] = class_prob
           
        # normalize class probabilities
        
        # to avoid NaN values, all the probabilities will remain the same
        if np.all(class_probs, 0):
            #print(f"found all probabilities 0 at image {i}")
            class_probs += 1
            
        total = np.sum(class_probs)
        
        posterior_probs = class_probs / total
        
        probs[i] = posterior_probs.reshape((5,))
        
    
    return probs
                

## Predict class

Predict that each sample belongs to the class with the highest class conditional probability

In [13]:
def predict_class(X, cond, priors):
    
    probs = predict_probabilities(X, cond, priors)
    
    # return class with highest probability for each sample
    return np.argmax(probs, axis=1)

## Test Naive Bayes performance

### Training set accuracy

In [14]:
n_bins = 5
conditionals_lda = get_conditionals(X_lda, y_train, n_bins)

In [15]:
predictions_lda = predict_class(X_lda, conditionals_lda, priors)

In [16]:
(np.sum(predictions_lda == y_train))/(X_lda.shape[0])

0.7598

### Test set accuracy

In [17]:
X_lda_test = X_test.dot(W)

In [18]:
predictions_lda_test = predict_class(X_lda_test, conditionals_lda, priors)

In [19]:
(np.sum(predictions_lda_test == y_test))/(X_lda_test.shape[0])

0.7098